## <a name="0.0"></a>Содержание:
* [0. Описание проекта, описание данных, импорт библиотек](#0.)
* [1. Загрузка и изучение данных:](#1.)
     - [1.1. Bar-plot по количеству совершенных полетов по каждой модели самолета:](#1.1.)
     - [1.2. Исследование по географии использования разных моделей самолетов в сентябре'18:](#1.2.)
       + [1.2.1. Загрузка файла с доп. данными](#1.2.1.)
       + [1.2.2. Получение координат каждого из населенных пунктов, присутствующих в датафрейме](#1.2.2.)
       + [1.2.3. Добавление столбца с расстоянием между точками перелета](#1.2.3.)
       + [1.2.4. Добавление столбца с расчетным временем перелета между двумя точками](#1.2.4.)
       + [1.2.5. Построение карты перелетов](#1.2.5.)
     - [1.3. Данные по среднему количеству рейсов в день по городам (с графиком))](#1.3.)
* [2. (Шаг 5) Проверка гипотезы о среднем спросе на билеты:](#2.)
     - [2.1. Загрузка данных](#2.1.)
     - [2.2. Формулировка гипотезы](#2.2.)
     - [2.3. Проверка гипотезы](#2.3.)
     - [2.4. Расчет доверительных интервалов по выборкам](#2.4.)     
* [3. Вывод](#3.)

## Описание данных<a name="0."></a>
<font size="2">([к содержанию](#0.0))</font><br/>

База данных об авиаперевозках:
Таблица <b>airports</b> — информация об аэропортах:
* <b>airport_code</b> — трёхбуквенный код аэропорта
* <b>airport_name</b> — название аэропорта
* <b>city</b> — город
* <b>timezone</b> — временная зона

Таблица <b>aircrafts</b> — информация об самолётах:
* <b>aircraft_code</b> — код модели самолёта
* <b>model</b> — модель самолёта
* <b>range</b> — количество самолётов

Таблица <b>tickets</b> — информация о билетах:
* <b>ticket_no</b> — уникальный номер билета
* <b>passenger_id</b> — персональный идентификатор пассажира
* <b>passenger_name</b> — имя и фамилия пассажира

Таблица <b>flights</b> — информация о рейсах:
* <b>flight_id</b> — уникальный идентификатор рейса
* <b>departure_airport</b> — аэропорт вылета
* <b>departure_time</b> — дата и время вылета
* <b>arrival_airport</b> — аэропорт прилёта
* <b>arrival_time</b> — дата и время прилёта
* <b>aircraft_code</b> – id самолёта

Таблица <b>ticket_flights</b> — стыковая таблица «рейсы-билеты»
* <b>ticket_no</b> — номер билета
* <b>flight_id</b> — идентификатор рейса

Таблица <b>festivals</b> — информация о фестивалях
* <b>festival_id</b> — уникальный номер фестиваля
* <b>festival_date</b> — дата проведения фестиваля
* <b>festival_city</b> — город проведения фестиваля
* <b>festival_name</b> — название фестиваля

Импортируем библиотеки:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

<a name="1."></a><br/>
<font size="4"><b>1. Загрузка и изучение данных</b></font>.<br/>
<font size="2">([к содержанию](#0.0))</font><br/>

In [2]:
model_flights = pd.read_csv('/datasets/query_1.csv')
city_flights = pd.read_csv('/datasets/query_3.csv')

Смотрим данные по количеству совершенных полетов по каждой модели самолета:

In [3]:
(model_flights
             .style.set_caption('Данные по количеству совершенных полетов по каждой модели самолета')
)

In [4]:
model_flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
model             8 non-null object
flights_amount    8 non-null int64
dtypes: int64(1), object(1)
memory usage: 256.0+ bytes


Типы данных корректны.

Пропуски в данных отсутствуют. <br/>
В столбце <b>model</b> данные имеют подходящий нам текстовый формат (object), значения в столбце <b>flights_amount</b> целочисленные - то, что нужно.

<a name="1.1."></a><br/>
<b>1.1. Bar-plot по количеству совершенных полетов по каждой модели самолета</b>.<br/>
<font size="2">([к содержанию](#0.0))</font><br/>

Строим график:

In [5]:
(model_flights.sort_values(by='flights_amount', ascending=False).reset_index(drop=True).style
                                        .bar(subset=['flights_amount'], color='#97F0AA', vmin=0, width=100)
                                        .set_caption('Данные по количеству совершенных полетов по каждой модели самолета')
)

Воспользуемся модулем <b>Bokeh</b> и всё-таки построим отдельный <b>barplot</b>:

In [6]:
!pip install bokeh

     |████████████████████████████████| 32.4MB 11.6MB/s eta 0:00:01
     |████████████████████████████████| 276kB 19.5MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 16.0MB/s eta 0:00:01
  Created wheel for bokeh: filename=bokeh-1.4.0-cp37-none-any.whl size=23689200 sha256=34d9a9dca0e021dd7c27648c8b7a1e358db79ac3955812050513edc6932cd984
  Stored in directory: /home/jovyan/.cache/pip/wheels/fb/f8/47/09700d9a19cbcbf0b7a3130690b75c0d6ff80fbda0b1774c7c
  Created wheel for PyYAML: filename=PyYAML-5.3-cp37-cp37m-linux_x86_64.whl size=44229 sha256=386861e95d23218b371c181c6fb7290489659f10423bc62681cc632bbaf997ec
  Stored in directory: /home/jovyan/.cache/pip/wheels/e4/76/4d/a95b8dd7b452b69e8ed4f68b69e1b55e12c9c9624dd962b191
Successfully built bokeh PyYAML


In [7]:
from bokeh.io import show, output_file, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool

In [8]:
output_notebook()  # для корректной работы в Jypiter Notebook

Loading BokehJS ...

Строим график:

In [9]:
model_flights_count = model_flights.copy()
model_flights_count = model_flights_count.sort_values(by='flights_amount', ascending=False)


airplanes = model_flights_count['model'].values
values = model_flights_count['flights_amount'].values
title = "Данные по количеству совершенных полетов по каждой модели самолета"

p = figure(x_range=airplanes, 
           plot_height = 350, 
           title=title, 
           tools="box_edit", 
           toolbar_location="right")

source = ColumnDataSource(model_flights_count)
p.vbar(x='model', top='flights_amount', width=0.9, source=source, color='#97F0AA', line_color="black")
p.add_tools(HoverTool(tooltips=[("Модель самолета", "@model"), ("Кол-во вылетов", "@flights_amount")]))
p.xaxis.major_label_orientation = 3.14/3.5
p.xaxis.major_label_text_font_style = "bold"

show(p)

При наведении мышкой доступны данные

3 модели воздушных судов, которые являются лидерами по частоте использования в России (за период сентябрь'18):
* <b>Cessna 208 Caravan</b> - 4557 перелетов - небольшие самолеты на 9-13 мест (используют а/к в Сибири);
* <b>Bombardier CRJ-200</b> - 4446 перелетов - самолет на 50 мест, летает Utair;
* <b>Sukhoi SuperJet-100</b> - 4185 перелетов - вместимость - 98 мест.

<a name="1.2."></a><br/>
<b>1.2. Исследование по географии использования разных моделей самолетов в сентябре'18</b>.<br/>
<font size="2">([к содержанию](#0.0))</font><br/>

Узнаем, в какой части России какая модель самолета чаще всего была задействована в сентябре 2018 года.<br/><br/>
Для этого доп. запросом SQL выгрузим данные из базы и загрузим датафрейм с базой в <b>Jupyter</b> Яндекс.Практикума

<a name="1.2.1."></a><br/>
<b>1.2.1. Загрузка файла с доп. данными</b>.<br/>
<font size="2">([к содержанию](#0.0))</font><br/>

Файл содержит данные по модели самолета, городу вылета, города прибытия и количеству перелетов данной модели самолета по такому направлению.<br/><br/>
Подгрузим его в проект:

In [10]:
cities_models = pd.read_csv('cities_model_dep_arr_all.csv', encoding='cp1251')
cities_models.columns = ['model', 'city_departure', 'city_arrival', 'flights_count']

In [11]:
cities_models.head().style.set_caption('Данные по количеству вылетов, модели самолетам и деталям перелета')

In [12]:
cities_models.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527 entries, 0 to 526
Data columns (total 4 columns):
model             527 non-null object
city_departure    527 non-null object
city_arrival      527 non-null object
flights_count     527 non-null int64
dtypes: int64(1), object(3)
memory usage: 16.6+ KB


Данные чистые, без пропусков, изменять типы столбцов нет необходимости.

<a name="1.2.2."></a><br/>
<b>1.2.2. Получение координат каждого из населенных пунктов, присутствующих в датафрейме</b>.<br/>
<font size="2">([к содержанию](#0.0))</font><br/>

Получаем список уникальных городов:

In [13]:
list_of_cities = list(set(cities_models['city_departure'].tolist() + cities_models['city_arrival'].tolist()))

Воспользуемся <b>Yandex Геокодер</b>'ом для получения координат каждого города.<br/><br/>
Запускаем функцию, которая вернет нам список <b>coordinates</b>, содержащий информацию о координатах по каждому городу:

In [14]:
# подгружаем токен
from yaml import safe_load

with open('config_Geocoder.yaml') as file:
    token_source = safe_load(file)
    
token_Geocoder = token_source['Yandex Maps']['token']

In [15]:
# запускаем функцию
import requests as r
url = 'https://geocode-maps.yandex.ru/1.x/?format=json&apikey={}&geocode='.format(token_Geocoder)
coordinates = []

for city in list_of_cities:
    if city == city: # чтоб не столкнуться с nan
        url_formatted = url + city
        response = r.get(url_formatted).json()
        data = response['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point'].get('pos')
        coordinate = (float(data.split()[1]), float(data.split()[0]))
        coordinates.append(coordinate)
    else:
        coordinates.append('')

<div style="background-color:lightblue">
<font size="4" color=#825B04><b>Комментарий от студента</b></font><br/>


<font size="2">Сразу после принятия проекта в работу старый api-код деактивирую и завожу новый.</font><br/>


Огромное спасибо за инструкцию по использованию в работе .yaml-файлов, <font color=#048205><b>отработал согласно рекомендации</b></font>.
</div>

Формируем датафрейм, чтобы посмотреть, что у нас получилось:

In [16]:
unique_cities_coordinates = pd.DataFrame({'Город':list_of_cities, 'Координаты':coordinates})
unique_cities_coordinates.head(10).style.set_caption('Данные город-координаты')

Проставим координаты в отдельных столбцах датафрейма <b>cities_models</b>:

Формируем словарь, чтобы потом добавить столбец 'coordinates' в основной датафрейм <b>cities_models</b>:

In [17]:
dict_unique_cities_coordinates = dict(pd.Series(unique_cities_coordinates['Координаты'].values,
                                index=unique_cities_coordinates['Город'].values))

In [18]:
cities_models['coordinates_dep'] = cities_models['city_departure'].map(dict_unique_cities_coordinates)
cities_models['coordinates_arr'] = cities_models['city_arrival'].map(dict_unique_cities_coordinates)

In [19]:
cities_models.head().style.set_caption('Данные по количеству вылетов, модели самолетам и деталям перелета')

<a name="1.2.3."></a><br/>
<b>1.2.3. Добавление столбца с расстоянием между точками перелета</b>.<br/>
<font size="2">([к содержанию](#0.0))</font><br/>

Для этого воспользуемся пакетом <b>geopy</b>.

In [20]:
!pip install geopy

     |████████████████████████████████| 102kB 1.7MB/s ta 0:00:011


Импортируем его и добавляем новый столбец <b>distance</b>, где по каждой строке указано расстояние между городом вылета и городом прибытия.

In [21]:
from geopy import distance
cities_models['distance'] = cities_models.apply(lambda x: distance.distance(
                                                    x['coordinates_dep'], x['coordinates_arr']).km, axis=1)

<a name="1.2.4."></a><br/>
<b>1.2.4. Добавление столбца с расчетным временем перелета между двумя точками</b>.<br/>
<font size="2">([к содержанию](#0.0))</font><br/>

Сначала проставим в новый столбец <b>flight_time</b> время перелета в минутах (заложим +15 минут на взлет и посадку + 10% к расстояния по прямой из-за обычной кривизны авиамаршрутов):

In [22]:
flight_time_dict = {'Cessna 208 Caravan': 344,
                    'Bombardier CRJ-200': 780,
                    'Sukhoi SuperJet-100': 828,
                    'Airbus A319-100': 840,
                    'Airbus A321-200': 840,
                    'Boeing 737-300': 780,
                    'Boeing 767-300': 851,
                    'Boeing 777-300': 950}


def get_flight_time(row):
    if row['model'] in flight_time_dict:
        velocity_mmin = flight_time_dict[row['model']] * 1000/60
    else:
        raise KeyError('Unknown airplane model!')
    
    return (row['distance']*1000 + 0.1*row['distance'])/velocity_mmin + 15
    
    
cities_models['flight_time'] = cities_models.apply(get_flight_time, axis=1)

In [23]:
cities_models.head().style.set_caption('Данные по количеству вылетов, модели самолетам и деталям перелета')

Переведем минуты в часы и минуты:

In [24]:
def min_to_hours_min(cell_minutes):
    hours = cell_minutes // 60
    minutes = round(round(cell_minutes) - hours * 60, -1)
    if minutes == 60:
        hours += 1
        minutes = 0
    result = "%d:%02d" % (hours, minutes)
    return result

cities_models['flight_time'] = cities_models['flight_time'].apply(min_to_hours_min)

In [25]:
cities_models.head().style.set_caption('Данные по количеству вылетов, модели самолетам и деталям перелета')

In [26]:
# cities_models['distance'] = cities_models['distance'].astype('int').map('{0:g}'.format)
# cities_models['distance'] = cities_models['distance'] + ' км'

Округлим данные в столбце <b>distance</b>:

In [27]:
cities_models['distance'] = cities_models['distance'].astype('int')

Для удобства разобьем данные по координатам на разные столбцы (широта, долгота):

In [28]:
cities_models['Lat_dep'] = cities_models['coordinates_dep'].apply(lambda x: x[0])
cities_models['Long_dep'] = cities_models['coordinates_dep'].apply(lambda x: x[1])
cities_models['Lat_arr'] = cities_models['coordinates_arr'].apply(lambda x: x[0])
cities_models['Long_arr'] = cities_models['coordinates_arr'].apply(lambda x: x[1])
cities_models = cities_models.drop(['coordinates_dep', 'coordinates_arr'], axis=1)

Опять же для удобства изменим порядок отображения столбцов датафрейма:

In [29]:
cities_models = cities_models[['model', 'city_departure', 'Lat_dep', 'Long_dep', 'city_arrival', 
                               'Lat_arr', 'Long_arr', 'flights_count', 'distance', 'flight_time']]

cities_models.head().style.set_caption('Данные по количеству вылетов, модели самолетам и деталям перелета')

<a name="1.2.5."></a><br/>
<b>1.2.5. Построение карты перелетов</b>.<br/>
<font size="2">([к содержанию](#0.0))</font><br/>

In [30]:
#конвертируем координаты в формат WEB MERCATOR
def wgs84_to_web_mercator(df, lat="Lat_dep", lon="Long_dep"):
    k = 6378137
    name_1 = "x_dep"
    name_2 = "y_dep"
    if name_1 in df.columns:
        name_1 = "x_arr"
        name_2 = "y_arr"
        lat = "Lat_arr"
        lon = "Long_arr"
    df[name_1] = df[lon] * (k * np.pi/180.0)
    df[name_2] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

wgs84_to_web_mercator(cities_models)
wgs84_to_web_mercator(cities_models)

model city_departure    Lat_dep    Long_dep city_arrival  \
0        Airbus A319-100         Абакан  53.721152   91.442387  Архангельск   
1        Airbus A319-100         Абакан  53.721152   91.442387       Москва   
2        Airbus A319-100        Анадырь  64.733115  177.508924       Москва   
3        Airbus A319-100        Анадырь  64.733115  177.508924    Хабаровск   
4        Airbus A319-100    Архангельск  64.539911   40.515753       Абакан   
..                   ...            ...        ...         ...          ...   
522  Sukhoi SuperJet-100      Челябинск  55.159897   61.402554       Москва   
523  Sukhoi SuperJet-100      Череповец  59.122612   37.903461  Новокузнецк   
524  Sukhoi SuperJet-100         Элиста  46.307743   44.269759       Москва   
525  Sukhoi SuperJet-100         Элиста  46.307743   44.269759          Уфа   
526  Sukhoi SuperJet-100         Якутск  62.027757  129.731235      Барнаул   

       Lat_arr    Long_arr  flights_count  distance flight_time         x_dep  \
0    64.539911   40.515753              4      3051        3:50  1.017932e+07   
1    55.753215   37.622504              9      3386        4:20  1.017932e+07   
2    55.753215   37.622504             13      6213        7:40  1.976020e+07   
3    48.480223  135.071917              5      3081        4:00  1.976020e+07   
4    53.721152   91.442387              4      3051        3:50  4.510193e+06   
..         ...         ...            ...       ...         ...           ...   
522  55.753215   37.622504             30      1498        2:00  6.835301e+06   
523  53.757547   87.136044              4      3021        3:50  4.219394e+06   
524  55.753215   37.622504             30      1148        1:40  4.928087e+06   
525  54.735147   55.958727             30      1248        1:40  4.928087e+06   
526  53.346785   83.776856              4      2835        3:40  1.444162e+07   

            y_dep         x_arr         y_arr  
0    7.117522e+06  4.510193e+06  9.488212e+06  
1    7.117522e+06  4.188118e+06  7.509444e+06  
2    9.538421e+06  4.188118e+06  7.509444e+06  
3    9.538421e+06  1.503614e+07  6.187122e+06  
4    9.488212e+06  1.017932e+07  7.117522e+06  
..            ...           ...           ...  
522  7.392961e+06  4.188118e+06  7.509444e+06  
523  8.206935e+06  9.699940e+06  7.124372e+06  
524  5.829803e+06  4.188118e+06  7.509444e+06  
525  5.829803e+06  6.229297e+06  7.310632e+06  
526  8.865727e+06  9.325997e+06  7.047403e+06  

[527 rows x 14 columns]

Смотрим карту городов и карту перелетов по каждой из модели самолетов в сентябре 2018г.:

In [31]:
import warnings
warnings.filterwarnings("ignore")

from bokeh.plotting import figure, show, output_file
from bokeh.tile_providers import get_provider, Vendors

from bokeh.layouts import layout, row, column
from bokeh.models.widgets import Tabs, Panel
from bokeh.plotting import figure

from bokeh.models import ColorBar, LogColorMapper, BasicTicker

output_file("tile.html")

# выбираем вид карты
tile_provider = get_provider(Vendors.CARTODBPOSITRON_RETINA)

# общий источник данных
flight_source = ColumnDataSource({
    'x_arr': cities_models['x_arr'].unique(),
    'y_arr': cities_models['y_arr'].unique(), 
    'city_arrival': cities_models['city_arrival'].unique(),
})

# источник данных для столиц
source_capitals = ColumnDataSource({
    'x_arr': cities_models[(cities_models['city_arrival']=='Москва') |
                           (cities_models['city_arrival']=='Санкт-Петербург')]['x_arr'].unique(),
    'y_arr': cities_models[(cities_models['city_arrival']=='Москва') |
                           (cities_models['city_arrival']=='Санкт-Петербург')]['y_arr'].unique(), 
    'city_arrival': cities_models[(cities_models['city_arrival']=='Москва') |
                           (cities_models['city_arrival']=='Санкт-Петербург')]['city_arrival'].unique(),
})

# палитра цветов
color_mapper = LogColorMapper(palette="Plasma256", low=10, high=cities_models['flights_count'].max())

# рисуем карту городов отправления/прибытия
p1 = figure(x_range=(1500000, 20000000), y_range=(5500000, 12000000),
           x_axis_type="mercator", y_axis_type="mercator", plot_width=980, plot_height=400, sizing_mode='scale_width')
p1.add_tile(tile_provider)

# все города
p1.circle('x_arr','y_arr', source=flight_source, fill_color="#92092C", size=12, 
          fill_alpha=0.5, line_width=1)

# столицы
p1.circle('x_arr','y_arr', source=source_capitals, fill_color="blue", size=20, 
          fill_alpha=0.5, line_width=1)

p1.add_tools(HoverTool(tooltips=[("Город", "@city_arrival")]))

# график статистики перелетов по модели
model_flights1 = model_flights.copy()
model_flights1 = model_flights1.sort_values(by='flights_amount', ascending=False)
airplanes = model_flights1['model'].values
values = model_flights1['flights_amount'].values
title = "Данные по количеству совершенных полетов по каждой модели самолета"

# выводим статистику по количеству перелетов по моделям самолетов
p2 = figure(x_range=airplanes, 
            plot_width=910,
            plot_height=400, 
            title=title, 
            tools="box_edit", 
            toolbar_location="right")

source = ColumnDataSource(model_flights1)
p2.vbar(x='model', top='flights_amount', width=0.9, source=source, color='#97F0AA', line_color="black")
p2.add_tools(HoverTool(tooltips=[("Модель самолета", "@model"), ("Кол-во вылетов", "@flights_amount")]))
p2.xaxis.major_label_orientation = 3.14/3.5
p2.xaxis.major_label_text_font_style = "bold"


# создаем наборы переменных/данных для отрисовки карты перелетов по моделям
ps = ['p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10']
list_of_models = ['Cessna 208 Caravan', 'Bombardier CRJ-200', 'Sukhoi SuperJet-100',
                  'Airbus A321-200', 'Boeing 737-300',
                  'Airbus A319-100', 'Boeing 767-300', 'Boeing 777-300']
layouts = ['l3', 'l4', 'l5', 'l6', 'l7', 'l8', 'l9', 'l10']
tabs = ['tab3', 'tab4', 'tab5', 'tab6', 'tab7', 'tab8', 'tab9', 'tab10']
tab_names = ['Cessna 208 Caravan', 'Bombardier CRJ-200', 'Sukhoi SuperJet-100',
                  'Airbus A321-200', 'Boeing 737-300', 'Airbus A319-100', 'Boeing 767-300', 'Boeing 777-300']
list_of_ps = list()

# рисуем карту перелетов по каждой модели самолета
for i in range(len(ps)):
    p_curr = ps[i]
    model_curr = list_of_models[i]
    
    # создаем пустую карту
    p_curr = figure(x_range=(1500000, 20000000), y_range=(5500000, 12000000),
           x_axis_type="mercator", y_axis_type="mercator", plot_width=980, plot_height=400, sizing_mode='scale_width')
    p_curr.add_tile(tile_provider)
    
    # источник для multi-lines
    source_lines1 = ColumnDataSource({
    'x_mult_dep': [cities_models[cities_models['model']==model_curr]['x_dep'].to_list()[0], 
               cities_models[cities_models['model']==model_curr]['x_arr'].to_list()[0]],
    'y_mult_dep': [cities_models[cities_models['model']==model_curr]['y_dep'].to_list()[0], 
               cities_models[cities_models['model']==model_curr]['y_arr'].to_list()[0]]
    })
    
    # источник для multi-lines - корр.
    cities_models1 = cities_models.copy()
    cities_models1 = cities_models1[cities_models1['model']==model_curr].reset_index(drop=True)

    
    source_lines = ColumnDataSource(dict
                (xs=[[cities_models1.loc[i, 'x_dep'], cities_models1.loc[i, 'x_arr']] for i in range(len(cities_models1))],
                 ys=[[cities_models1.loc[i, 'y_dep'], cities_models1.loc[i, 'y_arr']] for i in range(len(cities_models1))],
                 city_arrival=cities_models1['city_arrival'],
                 city_departure=cities_models1['city_departure'],
                 flights_count=cities_models1['flights_count'],
                 distance=cities_models1['distance'],
                 flight_time = cities_models1['flight_time']))
    
    # источник для активных городов
    source_active_cities = ColumnDataSource({
    'x_dep': cities_models1['x_dep'],
    'y_dep': cities_models1['y_dep'], 
    'x_arr': cities_models1['x_arr'],
    'y_arr': cities_models1['y_arr'], 
    'city_arrival': cities_models1['city_arrival'],
    'city_departure': cities_models1['city_departure'],
    'flights_count': cities_models1['flights_count'],   
    })
    
    
    # отрисовываем все города
    r1 = p_curr.circle('x_arr','y_arr', source=flight_source, fill_color="#92092C", size=14, 
                  fill_alpha=0.5, line_width=0.5)
    # отмечаем активные города
    r2 = p_curr.circle('x_dep','y_dep', source=source_active_cities, fill_color="black", size=14, 
                  fill_alpha=0.8, line_width=0.5)
    r3 = p_curr.circle('x_arr','y_arr', source=source_active_cities, fill_color="black", size=14, 
                  fill_alpha=0.8, line_width=0.5)
    # отмечаем столицы
    r4 = p_curr.circle('x_arr','y_arr', source=source_capitals, fill_color="blue", size=20, 
          fill_alpha=0.5, line_width=1)
    
    # рисуем линии перелетов
    r5 = p_curr.multi_line('xs', 'ys', color={'field': 'flights_count', 'transform': color_mapper}, source=source_lines)

    tooltips_active_cities = [("Город вылета", "@city_departure"), ("Город прибытия", "@city_arrival"), 
                              ("Количество перелетов", "@flights_count")]
    
    tooltips_lines = [("Город вылета", "@city_departure"), ("Город прибытия", "@city_arrival"), 
                      ("Количество перелетов", "@flights_count"), ("Расстояние, км", "@distance"),
                      ("Ориентировочное время перелета", "@flight_time")]
    
    color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0))
    
    p_curr.add_layout(color_bar, 'right')
    
    # добавляем HoverTool для всех городов
    p_curr.add_tools(HoverTool(renderers=[r1], tooltips=[("Город", "@city_arrival")]))
    # добавляем HoverTool для активных городов
    p_curr.add_tools(HoverTool(renderers=[r2, r3], tooltips=tooltips_active_cities))
    # добавляем HoverTool для линий
    p_curr.add_tools(HoverTool(renderers=[r5], tooltips=tooltips_lines))
    # пополняем список фигур
    list_of_ps.append(p_curr)
    
# создаем layout'ы по каждому из графиков
l1 = layout([[p1]])
l2 = layout([[p2]])
for p_count in range(len(ps)):
    layouts[p_count] = layout([[list_of_ps[p_count]]])
    
# формируем закладки для каждого из графиков
tab1 = Panel(child=l1, title='Карта городов прилета/вылета')
tab2 = Panel(child=l2, title='Статистика по перелетам по модели')

for tab_count in range(len(tabs)):
    tabs[tab_count] = Panel(child=layouts[tab_count], title=tab_names[tab_count])

# разбиваем графики (закладки) на 2 группы для отображения в 2 разных окнах
tabs1 = Tabs(tabs=[tab2, tab1])
tabs2 = Tabs(tabs=[*tabs])

# выводим 2 набора графиков на экран один по другим
show(column(tabs1, tabs2))

Таким образом, можно сделать вывод:

* <b>Cessna 208 Caravan</b> - небольшие самолеты на 9-13 мест, летают по небольшому количеству маршрутво и сугубо на короткие расстояния, обслуживают только направления с малым пассажиропотоком. И это логично, вместимость-то малая;
* <b>Bombardier CRJ-200</b> - самолет на 50 мест, география намного более объемная, чем у "Cessna". Летают на средние расстояния, присутствуют высокозагруженные направления (кроме Санкт-Петербург-Москва и обратно). Это Utair;
* <b>Sukhoi SuperJet-100</b> - вместимость - 98 мест. География похожа на географию <b>Bombardier</b>, а по частоте перелетов по маршруту Москва-Брянск и обратно можно угадать <b>S7</b>;
* <b>Airbus A321-200</b> - вместимость - 188 мест. География неразнообразна, основное направление - Санкт-Петербург-Москва (и обратно). Похоже на тот же <b>S7</b> - только они обеспечивают прямой перелет Санкт-Петербург-Иркутск (и обратно; хотя по данным мы не можем гарантировать, что перелеты прямые);
* <b>Boeing 737-300</b> - вместимость - 188 мест. География не сильно разнообразна. В основном летают из Москвы, есть одно направление из Санкт-Петербурга и локальные перелеты на юге;
* <b>Airbus A319-100</b> - рассчитан на 124 пассажиров. В основном летает за Урал;
* <b>Boeing 767-300</b> - вместимость - 269 мест. География узкая, единственный перелет из Санкт-Петербурга - это в Хабаровск, и несколько направлений из Москвы (в том числе и на Дальний Восток);
* <b>Boeing 777-300</b> - вместимость - 407 пассажиров. И удивительно, но география слабая, и перелеты на малые расстояния, видимо, у а/к всего один борт, если они его испсользуют таким образом.

<a name="1.3."></a><br/>
<b>1.3. Данные по среднему количеству рейсов в день по городам (с графиком)</b>.<br/>
<font size="2">([к содержанию](#0.0))</font><br/>

In [32]:
(city_flights.head(10)
             .style.set_caption('Данные по среднему количеству рейсов в день по городам')
)

In [33]:
city_flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 2 columns):
city               101 non-null object
average_flights    101 non-null float64
dtypes: float64(1), object(1)
memory usage: 1.7+ KB


Типы данных корректны. Округлим до 2 знаков после запятой столбец <b>average_flights</b>:

In [34]:
city_flights['average_flights'] = city_flights['average_flights'].round(2)

Посмотрим на датафрейме <b>bar-plot</b> по среднему количеству вылетов в каждом из городов (топ-10 выделим красным цветом):

In [35]:
city_flights = city_flights.sort_values(by='average_flights', ascending=False).reset_index(drop=True)
top10_subset = pd.IndexSlice[city_flights.iloc[:10,1].index, 'average_flights']
(city_flights.style.bar(subset=['average_flights'], color='#97F0AA')
                   .bar(subset=top10_subset, color='#ff5050', vmin=0)
                   .set_caption('Данные по среднему количеству рейсов в день по городам')
)

Опять воспользуемся модулем <b>Bokeh</b> и нарисуем <b>barplot</b>, выделив топ-10 (также, <b>при наведении мышкой</b> на каждый столбец выдается информация с городом и средним количеством вылетов в день - учитывая плотность графика, это актуально):

In [36]:
from bokeh.transform import linear_cmap
# from bokeh.palettes import Spectral6

city_flights_vbar = city_flights.copy()
city_flights_vbar = city_flights_vbar.sort_values(by='average_flights', ascending=False)

for i in range(len(city_flights_vbar['city'][:10])):
    city_flights_vbar.loc[i, 'city'] = str(i+1) + ' ' + city_flights_vbar.loc[i, 'city']

cities = city_flights_vbar['city'].values
avg_flights_values = city_flights_vbar['average_flights'].sort_values(ascending=False).values
title = "Данные по среднему количеству рейсов в день по городам"

tools = "save, pan, box_zoom, reset, wheel_zoom"

p1 = figure(x_range=cities, 
            plot_width=1600,
            plot_height=450, 
            title=title, 
            tools=tools, 
            toolbar_location="left")


    
source = ColumnDataSource(city_flights_vbar)
palette = ['#97F0AA', '#ff5050']
mapper = linear_cmap(field_name='average_flights', palette=palette, low=avg_flights_values[10], high=avg_flights_values[9])
p1.vbar(x='city', top='average_flights', width=0.9, source=source, color=mapper, line_color="black")
p1.add_tools(HoverTool(tooltips=[("Город", "@city"), ("Среднее кол-во вылетов в день", "@average_flights{1.11}")]))
p1.xaxis.major_label_orientation = 3.14/3.5
p1.xaxis.major_label_text_font_style = "bold"

show(p1)

<a name="2."></a><br/>
<font size="4"><b>2. (Проверка гипотезы о среднем спросе на билеты</b></font>.<br/>
<font size="2">([к содержанию](#0.0))</font><br/>

<a name="2.1."></a><br/>
<b>2.1. Загружаем датафрейм с фестивалями</b>.<br/>
<font size="2">([к содержанию](#0.0))</font><br/>

In [37]:
festivals = pd.read_csv('/datasets/query_last.csv')

In [38]:
print('Количество строк - {}'.format(festivals.shape[0]))

Количество строк - 10


In [39]:
festivals.style.set_caption('Данные по фестивалям')

In [40]:
festivals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
week_number      10 non-null int64
ticket_amount    10 non-null int64
festival_week    3 non-null float64
festival_name    3 non-null object
dtypes: float64(1), int64(2), object(1)
memory usage: 448.0+ bytes


Данные нормальные, с ними можно работать. NaN'ы нам не помешают, а наоборот, даже помогут при формировании датафреймов с днями, когда был фестиваль и когда не был.

<a name="2.2."></a><br/>
<b>2.2. Формулировка гипотезы</b>.<br/>
<font size="2">([к содержанию](#0.0))</font><br/>

<b>Гипотеза: «Средний спрос на билеты во время фестивалей не отличается от среднего спроса на билеты в обычное время».</b>

Для проверки гипотезы будем использовать t-критерий Стьюдента, т.к. выборки независимы между собой.<br/>
Определим пороговое значение alpha = 0.05.

Н0: (предположение о том, что между данными у нас нет связи) <b>Средний спрос на билеты во время фестивалей не отличается от среднего спроса на билеты в обычное время</b><br/>
Н1: <b>Средний спрос на билеты во время фестивалей отличается от среднего спроса на билеты в обычное время</b>

Формируем 2 датафрейма: 1 - с неделями, когда были фестивали; 2 - с неделями, когда фестивалей не проводилось:

In [41]:
festival_days = festivals[festivals['festival_week'].notnull()]['ticket_amount']
not_festival_dats = festivals[festivals['festival_week'].isnull()]['ticket_amount']

<a name="2.3."></a><br/>
<b>2.3. Проверка гипотезы</b>.<br/>
<font size="2">([к содержанию](#0.0))</font><br/>

In [42]:
from scipy.stats import ttest_ind
import scipy.stats as st

Установим параметр <b>equal_var = False</b>, т.к. пусть и выборки независимы между собой, но мы не уверены, что они имеют одинаковую дисперсию:

In [43]:
stat, p = ttest_ind(festival_days, not_festival_dats, equal_var=False)
print('t = {}\np-value = {:.10e}'.format(stat, p))
print('\nПороговое значение alpha = 0.05')
print('\nИтог - {}'.format('отклоняем нулевую гипотезу' if p <= 0.05 else 'оставляем в силе нулевую гипотезу'))

t = -1.1248513751916296
p-value = 3.7743249317e-01

Пороговое значение alpha = 0.05

Итог - оставляем в силе нулевую гипотезу


Получается, что верно следующее утверждение:<br/>
<b>Средний спрос на билеты во время фестивалей не отличается от среднего спроса на билеты в обычное время.</b>

<a name="2.4."></a><br/>
<b>2.4. Расчет доверительных интервалов по выборкам</b>.<br/>
<font size="2">([к содержанию](#0.0))</font><br/><br/>

Ради интереса, посмотрим на доверительный интервал по каждой из выборок:

In [44]:
import statsmodels.stats.api as sms

In [46]:
print('Доверительный интервал по данным из выборки по дням, когда фестиваль проходил - {}'.
      format(sms.DescrStatsW(festival_days).tconfint_mean()))
print('Доверительный интервал по данным из выборки по дням, когда фестиваль не проходил - {}'.
      format(sms.DescrStatsW(not_festival_dats).tconfint_mean()))

Доверительный интервал по данным из выборки по дням, когда фестиваль проходил - (37693.65924921319, 59631.67408412014)
Доверительный интервал по данным из выборки по дням, когда фестиваль не проходил - (51409.764223837876, 51651.950061876414)


<a name="3."></a><br/>
<font size="4"><b>3. Вывод</b></font>.<br/>
<font size="2">([к содержанию](#0.0))</font><br/>

Мы сделали следующее:
* изучили данные по количеству перелетов за сентябрь 2018г. по каждой модели самолета, выяснили, что самыми используемыми моделями самолета (в сентябре 2018г.) являются <b>Cessna 208 Caravan</b>,  <b>Bombardier CRJ-200</b> и <b>Sukhoi SuperJet-100</b>;
* построили <b>карту перелетов</b> и оценили географию применения каждой модели самолета и их задействованность;
* пришли к выводу, что <b>в дни фестивалей средний спрос на авиабилеты не отличается от спрос на авиабилеты в те дни, когда фестивали не проводятся</b>.